# Deep Neural Networks (DNN) Model Development

## Preparing Packages

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from sklearn import metrics
from numpy import genfromtxt
from scipy import stats
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
from keras.callbacks import Callback
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
import keras
from keras.layers import Dense, Flatten, Reshape,Dropout
from keras.layers import Conv2D, MaxPooling2D, LSTM
from keras.models import Sequential
from sklearn.model_selection import train_test_split
import timeit #package for recording the model running time
import time
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, Conv3D, MaxPooling3D, Reshape, BatchNormalization, MaxPooling2D
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam 
from keras import backend as K
from sklearn.metrics import fbeta_score
from sklearn.model_selection import KFold,StratifiedKFold,ShuffleSplit,StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn import  preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,f1_score,accuracy_score

## Preparing Functions

In [ ]:
def win_seg(data,windowsize,overlap):#function for overlap segmentation
    length=int((data.shape[0]*data.shape[1]-windowsize)/(windowsize*overlap)+1)
    newdata=np.empty((length,windowsize, data.shape[2],1))
    data_dim=data.shape[2]
    layers=data.shape[3]
    data=data.reshape(-1,data_dim,layers)
    for i in range(0,length) :
        start=int(i*windowsize*overlap)
        end=int(start+windowsize)
        newdata[i]=data[start:end]
    return newdata
def lab_vote(data,windowsize):
    y_data=data.reshape(-1,windowsize,1,1)
    y_data=win_seg(y_data,windowsize,0.5)
    y_data=y_data.reshape(y_data.shape[0],y_data.shape[1],y_data.shape[2])
    y_data=stats.mode(y_data,axis=1)
    y_data=y_data.mode
    y_data=y_data.reshape(-1,1)
    y_data=np.float64(keras.utils.to_categorical(y_data))
    return y_data
def lab_vote_cat(data,windowsize): # non one-hot coding
    y_data=data.reshape(-1,windowsize,1,1)
    y_data=win_seg(y_data,windowsize,0.5)
    y_data=y_data.reshape(y_data.shape[0],y_data.shape[1],y_data.shape[2])
    y_data=stats.mode(y_data,axis=1)
    y_data=y_data.mode
    y_data=y_data.reshape(-1,1)
    return y_data
def write_csv(data):
    a = np.asarray(data)
    a.tofile('check.csv',sep=',',format='%10.5f')
def average(lst): 
    a = np.array(lst)
    return np.mean(a)
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []
    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()
    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

## Convolutional LSTM Model Development

In [ ]:
#loading the training and testing data
os.chdir("...") #changing working directory
buffer = np.float64(preprocessing.scale(genfromtxt('S3_X.csv', delimiter=','))) # using S3 as an example
x_data=buffer.reshape(-1,40,30,1)
x_data=win_seg(x_data,40,0.5) # data segmentation with 0.5 overlap
#majority vote on training label
buffer = np.float64(genfromtxt('S3_Y.csv', delimiter=','))-1 #0 based index
y_data=lab_vote(buffer,40)
y_data2=lab_vote_cat(buffer,40) # for stratification purposes
#five round Stratified Random Shuffle
SRS=StratifiedShuffleSplit(n_splits=5, test_size=0.1, random_state=42) #split the train and test by 9:1
#model evaluation metrics
acc_score=list()
f_score=list()
eopch_time_record=list()
oper_time_record=list()
i=0
for train_index, test_index in SRS.split(x_data,y_data):
    X_train, X_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]
    #split the train data into training (training the model) and validation (tuning hypeparameters) by 8:2
    X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20)
    #setup model parameters
    data_dim = X_train.shape[2] #y of 2D Motion Image
    timesteps = X_train.shape[1] #x of 2D Motion Image
    num_classes = y_train.shape[1]
    batchsize=300
    epcoh=300
    #build model
    model = Sequential()
    #five convolutional layers as an exmaple, adjust the convolutional layer depth if needed
    model.add(Conv2D(64, kernel_size=(5, 30), strides=(1, 1),padding='same',
                 activation='tanh',input_shape=(timesteps, data_dim,1)))
    model.add(Conv2D(64, kernel_size=(5, 30), strides=(1, 1),padding='same',
                 activation='tanh'))
    model.add(Conv2D(64, kernel_size=(5, 30), strides=(1, 1),padding='same',
                 activation='tanh'))
    model.add(Conv2D(64, kernel_size=(5, 30), strides=(1, 1),padding='same',
                 activation='tanh'))
    model.add(Conv2D(64, kernel_size=(5, 30), strides=(1, 1),padding='same',
                 activation='tanh'))
    #turn the multilayer tensor into single layer tensor
    model.add(Reshape((40, -1),input_shape=(40,30,64)))
    model.add(Dropout(0.5)) #add dropout layers for controlling overfitting
    model.add(LSTM(128, return_sequences=True, input_shape=(40, 1920)))  # returns a sequence of vectors
    model.add(Dropout(0.5)) #add dropout layers for controlling overfitting
    model.add(LSTM(128))  # return a single vector
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),metrics=['accuracy',f1])
    checkpointer = ModelCheckpoint(filepath="2D_CNN5_LSTM_checkpoint(F1)_sss_%s.h5" % i, monitor='val_f1',verbose=1, mode='max', save_best_only=True)
    time_callback = TimeHistory() #record the model training time for each epoch
    callbacks_list = [checkpointer,time_callback] 
    train_history=model.fit(X_training, y_training,
              batch_size=batchsize, epochs=epcoh,callbacks=callbacks_list,
              validation_data=(X_validation, y_validation))
    eopch_time=time_callback.times
    eopch_time_record.append(eopch_time) #record the traing time of each epoch
    CNN_LSTM_model=load_model("2D_CNN5_LSTM_checkpoint(F1)_sss_%s.h5" % i, custom_objects={'f1': f1})
    #model operation and timing
    start=timeit.default_timer()
    y_pred=CNN_LSTM_model.predict(X_test)
    stop=timeit.default_timer()
    oper_time=stop-start
    oper_time_record.append(oper_time)
    #check the model test result
    y_pred=CNN_LSTM_model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    Y_test=np.argmax(y_test, axis=1)
    acc_score.append(accuracy_score(Y_test, y_pred)) # Evaluation of accuracy
    f_score.append(f1_score(Y_test, y_pred,average='macro')) # Evaluation of F1 score
    print("This is the", i+1,  "out of ",5, "Shuffle")
    i+=1
    del model #delete the model for retrain the neural network from scrach, instead of starting from trained model
    
# record performance
performance=pd.DataFrame(columns=['Acc_score','Macro_Fscore','Average_Epoch','Average_Run'])
performance['Acc_score']=acc_score
performance['Macro_Fscore']=f_score
performance['Average_Epoch']=average(eopch_time_record)
performance['Average_Run']=average(oper_time_record)
performance.to_csv("2DConv5LSTM_Performance_sss_test.csv")

## Baseline LSTM Model Development

In [ ]:
acc_score=list()
f_score=list()
eopch_time_record=list()
oper_time_record=list()
#loading data
buffer = np.float64(preprocessing.scale(genfromtxt('S3_X.csv', delimiter=',')))
x_data=buffer.reshape(-1,40,30,1)
x_data=win_seg(x_data,40,0.5) # data segmentation with 0.5 overlap
x_data=x_data.reshape(x_data.shape[0],x_data.shape[1],x_data.shape[2]) #reshape the dataset as LSTM input shape
#majority vote on training label
buffer = np.float64(genfromtxt('S3_Y.csv', delimiter=','))-1 #0 based index
y_data=lab_vote(buffer,40)
i=0
for train_index, test_index in SRS.split(x_data,y_data):
    X_train, X_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]
    #split the train data into training (training the model) and validation (tuning hypeparameters) by 8:2
    X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20)
    #setup model parameters
    data_dim = X_train.shape[2] #y of figure
    timesteps = X_train.shape[1] #x of figure
    num_classes = y_train.shape[1]
    batchsize=300
    epcoh=300
    #Build Model
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(timesteps, data_dim)))   # returns a sequence of vectors of dimension 64
    model.add(Dropout(0.5)) #add dropout layers for controlling overfitting
    model.add(LSTM(128))  # return a single vector of dimension 64
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),metrics=['accuracy',f1])
    checkpointer = ModelCheckpoint(filepath='LSTM_checkpoint(F1)_sss_%s.h5' % i, monitor='val_f1',verbose=1,mode='max', save_best_only=True)
    time_callback = TimeHistory() #record the model training time for each epoch
    callbacks_list = [checkpointer,time_callback] 
    model.fit(X_training, y_training,
              batch_size=batchsize, epochs=epcoh,callbacks=callbacks_list,
              validation_data=(X_validation, y_validation))
    eopch_time=time_callback.times
    eopch_time_record.append(eopch_time) #record the traing time of each epoch
    LSTM_model=load_model('LSTM_checkpoint(F1)_sss_%s.h5' % i,custom_objects={'f1': f1})
    #model operation and timing
    start=timeit.default_timer()
    y_pred=LSTM_model.predict(X_test)
    stop=timeit.default_timer()
    oper_time=stop-start
    oper_time_record.append(oper_time)
    #check the model test result
    y_pred = np.argmax(y_pred, axis=1)
    Y_test=np.argmax(y_test, axis=1)
    acc_score.append(accuracy_score(Y_test, y_pred))
    f_score.append(f1_score(Y_test, y_pred,average='macro'))
    print("This is the", i+1,  "out of ",5, "Shuffle")
    del model #delete the model for retrain the neural network from scrach, instead of starting from trained model
    i+=1
# record performance
performance=pd.DataFrame(columns=['Acc_score','Macro_Fscore','Average_Epoch','Average_Run'])
performance['Acc_score']=acc_score
performance['Macro_Fscore']=f_score
performance['Average_Epoch']=average(eopch_time_record)
performance['Average_Run']=average(oper_time_record)
performance.to_csv("LSTM_Performance_sss_test.csv")

## Baseline CNN Model

In [ ]:
acc_score=list()
f_score=list()
eopch_time_record=list()
oper_time_record=list()
i=0
for train_index, test_index in SRS.split(x_data,y_data):
    X_train, X_test = x_data[train_index], x_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]
    #split the train data into training (training the model) and validation (tuning hypeparameters) by 8:2
    X_training, X_validation, y_training, y_validation = train_test_split(X_train, y_train, test_size=0.20)
    #setup model parameters
    data_dim = X_train.shape[2] #y of figure
    timesteps = X_train.shape[1] #x of figure
    num_classes = y_train.shape[1]
    batchsize=300
    epcoh=300
    #Build Model
    model = Sequential()
    model.add(Conv2D(64, kernel_size=(5, 30), strides=(1, 1),padding='same',
                 activation='tanh',input_shape=(timesteps, data_dim,1)))
    model.add(Conv2D(64, kernel_size=(5, 30), strides=(1, 1),padding='same',
                 activation='tanh'))
    model.add(Conv2D(64, kernel_size=(5, 30), strides=(1, 1),padding='same',
                 activation='tanh'))
    model.add(Conv2D(64, kernel_size=(5, 30), strides=(1, 1),padding='same',
                 activation='tanh'))
    model.add(Conv2D(64, kernel_size=(5, 30), strides=(1, 1),padding='same',
                 activation='tanh'))
    model.add(Flatten())
    model.add(Dropout(0.5)) #add dropout layers for controlling overfitting
    model.add(Dense(128, activation='tanh'))
    model.add(Dropout(0.5)) #add dropout layers for controlling overfitting
    model.add(Dense(128, activation='tanh'))
    model.add(Dense(num_classes, activation='softmax'))#second flat fully connected layer for softmatrix (classification)
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),metrics=['accuracy',f1])
    checkpointer = ModelCheckpoint(filepath='2D_CNN_checkpoint(F1)_sss_%s.h5' % i, monitor='val_f1',mode='max',verbose=1, save_best_only=True)
    time_callback = TimeHistory() #record the model training time for each epoch
    callbacks_list = [checkpointer,time_callback] 
    model.fit(X_training, y_training,
              batch_size=batchsize, epochs=epcoh,callbacks=callbacks_list,
              validation_data=(X_validation, y_validation))
    eopch_time=time_callback.times
    eopch_time_record.append(eopch_time) #record the traingtime of each epoch
    CNN_model=load_model('2D_CNN_checkpoint(F1)_sss_%s.h5' % i, custom_objects={'f1': f1})
    #model operation and timing
    start=timeit.default_timer()
    y_pred=CNN_model.predict(X_test)
    stop=timeit.default_timer()
    oper_time=stop-start
    oper_time_record.append(oper_time)
    #check the model test result
    y_pred = np.argmax(y_pred, axis=1)
    Y_test=np.argmax(y_test, axis=1)
    acc_score.append(accuracy_score(Y_test, y_pred))
    f_score.append(f1_score(Y_test, y_pred,average='macro'))
    print("This is the", i+1,  "out of ",5, "Shuffle")
    del model #delete the model for retrain the neural network from scrach, instead of starting from trained model
    i+=1
    
# record performance
import pandas as pd
performance=pd.DataFrame(columns=['Acc_score','Macro_Fscore','Average_Epoch','Average_Run'])
performance['Acc_score']=acc_score
performance['Macro_Fscore']=f_score
performance['Average_Epoch']=average(eopch_time_record)
performance['Average_Run']=average(oper_time_record)
performance.to_csv("2DConv_Performance_sss_test.csv")

# Benchmark Machine Learing-based Model Development

## Packages Preparation

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import  preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,f1_score,accuracy_score
import timeit
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification

## Functions Preparation

In [ ]:
def win_seg(data,windowsize,overlap):#function for overlap segmentation
    length=int((data.shape[0]*data.shape[1]-windowsize)/(windowsize*overlap)+1)
    newdata=np.empty((length,windowsize, data.shape[2],1))
    data_dim=data.shape[2]
    layers=data.shape[3]
    data=data.reshape(-1,data_dim,layers)
    for i in range(0,length) :
        start=int(i*windowsize*overlap)
        end=int(start+windowsize)
        newdata[i]=data[start:end]
    return newdata
def lab_vote(data,windowsize):
    y_data=data.reshape(-1,windowsize,1,1)
    y_data=win_seg(y_data,windowsize,0.5)
    y_data=y_data.reshape(y_data.shape[0],y_data.shape[1],y_data.shape[2])
    y_data=stats.mode(y_data,axis=1)
    y_data=y_data.mode
    y_data=y_data.reshape(-1,1)
    y_data=np.float64(keras.utils.to_categorical(y_data))
    return y_data
def lab_vote_cat(data,windowsize): # non one-hot coding
    y_data=data.reshape(-1,windowsize,1,1)
    y_data=win_seg(y_data,windowsize,0.5)
    y_data=y_data.reshape(y_data.shape[0],y_data.shape[1],y_data.shape[2])
    y_data=stats.mode(y_data,axis=1)
    y_data=y_data.mode
    y_data=y_data.reshape(-1,1)
    return y_data
def preparation(dataset):
    x_data=preprocessing.scale(pd.read_csv(dataset).iloc[:,1:]) #Column-wise normalization
    y_data=pd.read_csv(dataset).iloc[:,0]
    X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.20, random_state=42)#split the data into train and test by 8:2
    return X_train, X_test, x_data,y_train, y_test, y_data
def TrainModels(X_train, X_test, y_train, y_test):
    # Time cost
    train_time=[]
    run_time=[]
    #SVM
    svm=SVC(gamma='auto',random_state=42)
    start = timeit.default_timer()
    svm.fit(X_train,y_train)
    stop = timeit.default_timer()
    train_time.append(stop - start)
    start = timeit.default_timer()
    svm_pre=pd.DataFrame(data=svm.predict(X_test))
    stop = timeit.default_timer()
    run_time.append(stop - start)
    #Naive Bayes
    nb=GaussianNB()
    start = timeit.default_timer()
    nb.fit(X_train,y_train)
    stop = timeit.default_timer()
    train_time.append(stop - start)
    start = timeit.default_timer()
    nb_pre=pd.DataFrame(data=nb.predict(X_test))
    stop = timeit.default_timer()
    run_time.append(stop - start)
    #KNN
    knn=KNeighborsClassifier(n_neighbors=7) # based on a simple grid search
    start = timeit.default_timer()
    knn.fit(X_train,y_train)
    stop = timeit.default_timer()
    train_time.append(stop - start)
    start = timeit.default_timer()
    knn_pre=pd.DataFrame(data=knn.predict(X_test))
    stop = timeit.default_timer()
    run_time.append(stop - start)
    #Decision Tree
    dt=DecisionTreeClassifier(random_state=42)
    start = timeit.default_timer()
    dt.fit(X_train,y_train)
    stop = timeit.default_timer()
    train_time.append(stop - start)
    start = timeit.default_timer()
    dt_pre= pd.DataFrame(data=dt.predict(X_test))
    stop = timeit.default_timer()
    run_time.append(stop - start)
    #Random Forest
    rf=RandomForestClassifier(n_estimators=100)
    start = timeit.default_timer()
    rf.fit(X_train,y_train)
    stop = timeit.default_timer()
    train_time.append(stop - start)
    start = timeit.default_timer()
    rf_pre=pd.DataFrame(data=rf.predict(X_test))
    stop = timeit.default_timer()
    run_time.append(stop - start)
    report = pd.DataFrame(columns=['Models','Accuracy','Macro F1','Micro F1','Train Time','Run Time'])
    report['Models']=modelnames
    for i in range(len(result.columns)):
        report.iloc[i,1]=accuracy_score(y_test, result.iloc[:,i])
        report.iloc[i,2]=f1_score(y_test, result.iloc[:,i],average='macro')
        report.iloc[i,3]=f1_score(y_test, result.iloc[:,i],average='micro')
        if i<len(train_time):
            report.iloc[i,4]=train_time[i]
            report.iloc[i,5]=run_time[i]
    return report

## Sliding Window Segmentation

In [ ]:
#loading the training and testing data
os.chdir("...") #changing working directory
buffer = np.float64(genfromtxt('S3_X.csv', delimiter=','))
x_data=buffer.reshape(-1,40,30,1)
x_data=win_seg(x_data,40,0.5) # data segmentation with 0.5 overlap
x_data=x_data.reshape(-1,40,30)
x_data_pd=x_data.reshape(-1,30)
x_data_pd = pd.DataFrame(data=x_data_pd)
adj_win=[i//40+1 for i in range(len(x_data_pd.iloc[:,0]))]
x_data_pd["adjwin"]=adj_win
x_data_pd.to_csv("S3_X_ML.csv")
#majority vote on training label
buffer = np.float64(genfromtxt('S3_Y.csv', delimiter=',')) #0 based index
y_data=lab_vote(buffer,40)
y_data2=lab_vote_cat(buffer,40) # for stratification purposes
y_data_pd = pd.DataFrame(data=y_data2)
y_data_pd.to_csv("S3_Y_ML.csv")

## Feature Selection Using Recursive Feature Elimination

In [ ]:
X, y = X_train, y_train
svc = SVC(kernel="linear")
rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(10),scoring='f1_macro')
rfecv.fit(X, y)
print("Optimal number of features : %d" % rfecv.n_features_)
#plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (nb of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
plt.show()
# Export the best features
sel_features=pd.DataFrame()
sel_features["label"]=y_test
fullfeatures=pd.read_csv("fullfeatures.csv")
names=list(fullfeatures.columns.values)[1:]
for index, val in enumerate(list(rfecv.support_)):
    if val:
        sel_features=pd.concat([sel_features,fullfeatures.iloc[:,index+1]],axis=1)
sel_features.to_csv("S3_Dataset_ML_SelectetedFeatures.csv")

## Test on Selected Features

In [ ]:
X_train, X_test, X_data,y_train, y_test, y_data=preparation("S3_Dataset_ML_SelectetedFeatures.csv")
sf = ShuffleSplit(n_splits=5, test_size=0.1, random_state=42) # Random Shuffle
SRS = StratifiedShuffleSplit(n_splits=5, test_size=0.1, random_state=42) # Stratified Shuffle
finalreport = pd.DataFrame(columns=['Models','Accuracy','Macro F1','Micro F1','Train Time','Run Time'])
for train_index, test_index in SRS.split(X_data, y_data):
    X_train, X_test = X_data[train_index], X_data[test_index]
    y_train, y_test = y_data[train_index], y_data[test_index]
    finalreport=finalreport.append(TrainModels(X_train, X_test, y_train, y_test))
finalreport.to_csv("S3_Dataset_ML_SelectetedFeatures_Evalucation.csv")